In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import asin,pi, sqrt
import matplotlib.patches as patches
import matplotlib
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,AutoMinorLocator, LogLocator)
from scipy.optimize import curve_fit
from plotUtilities import *

In [ ]:
data_file_LF  = ['LF_M0.03_v0.15c_1.5d.dat', 'LF_M0.03_v0.15c_2.5d.dat', 'LF_M0.03_v0.15c_3.5d.dat', 'LF_M0.03_v0.15c_5.5d.dat']
data_file_LR  = ['LR_M0.03_v0.15c_1.5d.dat', 'LR_M0.03_v0.15c_2.5d.dat', 'LR_M0.03_v0.15c_3.5d.dat', 'LR_M0.03_v0.15c_5.5d.dat']

In [ ]:
OPAC_LF    = []   # liste des opac pour tous les temps 
OPAC_LR    = []
EPOCH = [1.5, 2.5, 3.5, 5.5]
for (i,epoch) in enumerate(EPOCH):
    data_LF = data_file_LF[i]
    data_LR = data_file_LR[i]
    opac_LF = np.genfromtxt(fname=data_LF, skip_header=2, dtype='float', usecols=(5))
    opac_LR = np.genfromtxt(fname=data_LR, skip_header=2, dtype='float', usecols=(5))
    OPAC_LF.append(opac_LF)
    OPAC_LR.append(opac_LR)
WaveLength = np.genfromtxt(fname=data_LF, skip_header=2, dtype='float', usecols=(0))

In [ ]:
################################################################## 
################################# FIT FONCTION ###################
################################################################## 
#lambda_0 =  WaveLength[index_ref]#4.0035000000E+03 
#opac_0   =  OPAC_LF[0][index_ref]
def fit_function(wave, alpha):# lambda_0, opac_0):
    #log_opac_fit = (np.log10(opac_0))*((wave/lambda_0)**(-alpha))
    LOG_Opac_Fit = LOG_kappa0 - alpha * (np.log10(wave) - LOG_lambda0 ) 
    return(LOG_Opac_Fit)
        
def fit_function_not_LOG(wave, alpha):
    opac_fit = opac_0*(wave/lambda_0)**(-alpha)
    return opac_fit
#########################################################################
################################# REBINNING ############################# 
#########################################################################


def rebin(a, *args):
    '''rebin ndarray data into a smaller ndarray of the same rank whose dimensions
    are factors of the original dimensions. eg. An array with 6 columns and 4 rows
    can be reduced to have 6,3,2 or 1 columns and 4,2 or 1 rows.
    example usages:
    >>> a=rand(6,4); b=rebin(a,3,2)
    >>> a=rand(6); b=rebin(a,2)
    '''
    shape = a.shape
    lenShape = len(shape)
    factor = (np.asarray(shape)/np.asarray(args))
    evList = ['a.reshape('] + \
             ['args[%d],int(factor[%d]),'%(i,i) for i in range(lenShape)] + \
             [')'] + ['.sum(%d)'%(i+1) for i in range(lenShape)] + \
             ['/int(factor[%d])'%i for i in range(lenShape)]
    
    #print(args[0], factor[0])
    #print(a.reshape(args[0], int(factor[0]),).sum(1))
    print (''.join(evList))
    return eval(''.join(evList))
    #return eval('a.reshape(args[0],factor[0],).sum(1)/factor[0]')

 #########################################################################
################################# LISSAGE ############################# 
#########################################################################

def lissage(Lx,Ly, p):
    
    '''Fonction qui débruite une courbe par une moyenne glissante
    sur 2P+1 points'''
    Lxout=[]
    Lyout=[]
    Lxout = Lx[p: -p]
    for index in range(p, len(Ly)-p):
        average = sum(Ly[index-p : index+p+1]) / (2*p + 1)
        Lyout.append(average)
        
    return Lxout,Lyout